## **Noise Models**

##### To test the functionality of the **NoiseModels** in isolation from the rest of the API. This is to confirm that noise is applied correctly to observations and that the result is valid and clipped to the observation space bounds.

#### Gaussian Model

In [51]:
import importlib
import Wrapper.src.wrapper_api.models.noise_model as nm
import Wrapper.src.wrapper_api.wrappers.noise_wrapper as nw
importlib.reload(nm)
importlib.reload(nw)
from Wrapper.src.wrapper_api.wrappers.noise_wrapper import NoiseWrapper
from Wrapper.src.wrapper_api.models.noise_model import GaussianNoise, LaplacianNoise, CustomNoise
from gymnasium import spaces
import numpy as np



### Dummy Env

In [52]:
from pettingzoo.utils import AECEnv, agent_selector


class DummyNWEnv(AECEnv):
    def __init__(self):
        super().__init__()
        self.possible_agents = ["agent_0", "agent_1"]
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.possible_agents)}
        self.agents = self.possible_agents[:]
        self.agent_selector = agent_selector(self.agents)
        self.agent_selection = self.agent_selector.next()
        self.observation_spaces = {
            agent: spaces.Box(0.0, 1.0, shape=(2,), dtype=np.float32) for agent in self.agents
        }
        self.action_spaces = {
            agent: spaces.Discrete(3) for agent in self.agents
        }

    def observe(self, current_agent):
        return np.array([0.4, 0.6], dtype=np.float32)

    def reset(self, seed=None, options=None):
        self.agents = self.possible_agents[:]
        self.agent_selector.reinit(self.agents)
        self.agent_selection = self.agent_selector.next()

    def step(self, action):
        self.agent_selection = self.agent_selector.next()

    def observation_space(self, current_agent):
        return self.observation_spaces[agent]

    def action_space(self, current_agent):
        return self.action_spaces[agent]

### Gaussian

In [56]:

# Create a Gaussian noise model
gaussian_model = GaussianNoise(mean=0.0, std=0.2)

# Wrap the DummyEnv with Gaussian noise
env = DummyNWEnv()
wrapped_env = NoiseWrapper(env, noise_model=gaussian_model)

# Reset and observe
wrapped_env.reset()
agent = wrapped_env.agent_selection
obs = wrapped_env.observe(agent)

print(f"Gaussian noisy observation for {agent}:", obs)


Gaussian noisy observation for agent_0: [0.48497882 0.56665063]


#### Laplacian Model

In [58]:

# Create a Laplacian noise model
laplace_model = LaplacianNoise(loc=0.0, scale=0.2)

# Wrap with Laplacian noise
env = DummyNWEnv()
wrapped_env = NoiseWrapper(env, noise_model=laplace_model)

# Reset and observe
wrapped_env.reset()
agent = wrapped_env.agent_selection
obs = wrapped_env.observe(agent)

print(f"Laplacian noisy observation for {agent}:", obs)


Laplacian noisy observation for agent_0: [0.33056852 0.71457137]


### Custom Noise Model
##### Allows definition of custom noise function

In [48]:
def custom_noise(cm_obs, space):
    return np.clip(cm_obs + 0.3, space.low, space.high)

cm_noise_model = CustomNoise(noise_fn=custom_noise)
env = DummyNWEnv()
wrapped_cm_env = NoiseWrapper(env, cm_noise_model)

wrapped_cm_env.reset()

agent = wrapped_cm_env.agent_selection
cm_obs = wrapped_cm_env.observe(agent)

print(f"Custom Noisy observation for {agent}:", cm_obs)



Custom Noisy observation for agent_0: [0.70000005 0.90000004]
